# Data Filtering

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_M = pd.read_csv('datos_analisis_por_comuna.csv') 
df_M = df_M.loc[:, ~df_M.columns.str.contains('^Unnamed')]
df_M.head()

,created_on,end_date,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,imput,...,MANRIQUE,PALMITAS,POPULAR,ROBLEDO,SAN ANTONIO DE PRADO,SAN CRISTOBAL,SAN JAVIER,SANTA CRUZ,SANTA ELENA,VILLA HERMOSA
0,9/14/2018,9/25/2018,11,-75.577025,6.211360,NaN,NaN,134,19.519293,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9/14/2018,9/25/2018,11,-75.572928,6.234350,NaN,NaN,588,19.701615,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9/14/2018,9/25/2018,11,-75.553186,6.237236,NaN,NaN,160,20.125429,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9/14/2018,9/25/2018,11,-75.594599,6.226078,NaN,NaN,90,19.583832,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9/14/2018,9/25/2018,11,-75.542267,6.205542,NaN,NaN,101,19.781657,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
# Separating the real values (not imputed)
df_real = df_M.loc[df_M['imput'] == 0]
print(df_real.shape)

# Counting NaN values of bedrooms and bathrooms respectively
print('Bedrooms nan values: ', df_real['bedrooms'].isna().sum())
print('Bathrooms nan values: ', df_real['bathrooms'].isna().sum())

(18775, 40)
Bedrooms nan values:  6199
Bathrooms nan values:  853


In [5]:
# Eliminate all the rows that contain a NaN value (provisional)
df_real = df_real.dropna(axis = 0)
df_real.shape

(12537, 40)

In [6]:
# Selection of households and apartments
esCasaoApto = df_real[['Apartamento', 'Casa']].apply(lambda x: False if (x[0] == 0 and x[1] == 0) else True, axis = 1)
df_real = df_real.loc[esCasaoApto].drop(columns = ['imput', 'Casa', 'Depósito', 'Finca', 'Local comercial', 'Lote', 
                                                   'Parqueadero', 'Oficina', 'Otro'])
df_real.columns

Index(['created_on', 'end_date', 'delta_time', 'lon', 'lat', 'bedrooms',
       'bathrooms', 'surface_total', 'price', 'Apartamento', 'ALTAVISTA',
       'ARANJUEZ', 'BELEN', 'BUENOS AIRES', 'CASTILLA', 'DOCE DE OCTUBRE',
       'EL POBLADO', 'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 'LAURELES',
       'MANRIQUE', 'PALMITAS', 'POPULAR', 'ROBLEDO', 'SAN ANTONIO DE PRADO',
       'SAN CRISTOBAL', 'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
       'VILLA HERMOSA'],
      dtype='object')

In [7]:
df_real['price'] = df_real['price'].apply(lambda x: np.exp(x))

# Comparing basic traits of each district

In [8]:
# Extracting the names of every district
columnas = df_real.columns.to_series()
nombres = columnas.loc['ALTAVISTA':]
nombres.reset_index(inplace = True, drop = True)
nombres

0                ALTAVISTA
1                 ARANJUEZ
2                    BELEN
3             BUENOS AIRES
4                 CASTILLA
5          DOCE DE OCTUBRE
6               EL POBLADO
7                 GUAYABAL
8               LA AMERICA
9            LA CANDELARIA
10                LAURELES
11                MANRIQUE
12                PALMITAS
13                 POPULAR
14                 ROBLEDO
15    SAN ANTONIO DE PRADO
16           SAN CRISTOBAL
17              SAN JAVIER
18              SANTA CRUZ
19             SANTA ELENA
20           VILLA HERMOSA
dtype: object

In [9]:
# Creating a list of dataframes containing the information of district
districts_df = [None] * nombres.shape[0]
d = {}
i = 0
for district in nombres:
    name = district.replace(' ', '_')
    vars()[name] = df_real.loc[df_real[district] == 1].loc[:,:'price'] 
    districts_df[i] = vars()[name]
    d[i] = name
    i += 1

In [10]:
# Describing each data set and extracting the data and storing it in a list
delta_time = [None] * nombres.shape[0]
bedrooms = [None] * nombres.shape[0]
bathrooms = [None] * nombres.shape[0]
surface_total = [None] * nombres.shape[0]
price = [None] * nombres.shape[0]
size = [None] * nombres.shape[0]

i = 0

while i < nombres.shape[0]:
    df = districts_df[i].describe().loc['mean'].drop(['lat', 'lon'])
    delta_time[i] = df[0]
    bedrooms[i] = df[1]
    bathrooms[i] = df[2]
    surface_total[i] = df[3]
    price[i] = df[4]
    size[i] = districts_df[i].shape[0]
    i += 1

In [11]:
# Turning it into a unique dataframe
districts = pd.DataFrame({
    'name': nombres.values.tolist(),
    'delta_time': delta_time, 
    'bedrooms': bedrooms, 
    'bathrooms': bathrooms, 
    'surface_total': surface_total,
    'price': price,
    'size': size
})
districts

,name,delta_time,bedrooms,bathrooms,surface_total,price,size
0,ALTAVISTA,68.730769,2.750000,2.057692,84.596154,3.468077e+08,52
1,ARANJUEZ,68.839080,4.068966,2.275862,149.310345,3.375081e+08,87
2,BELEN,66.717514,3.378531,2.674200,137.879473,4.365178e+08,1062
3,BUENOS AIRES,87.429213,3.804494,2.997753,278.516854,6.795728e+08,445
4,CASTILLA,72.098039,3.705882,2.568627,130.647059,3.837451e+08,51
5,DOCE DE OCTUBRE,36.400000,3.600000,3.000000,136.200000,4.380000e+08,5
6,EL POBLADO,75.208403,3.297059,3.497059,386.276891,8.266809e+08,2380
7,GUAYABAL,79.724280,3.438272,2.823045,201.629630,5.938528e+08,486
8,LA AMERICA,69.788686,3.723794,2.607321,143.811980,3.978516e+08,601
9,LA CANDELARIA,59.983752,3.691285,2.927622,176.428360,5.508242e+08,677


# Plotting

In [12]:
import plotly
from plotly.offline import init_notebook_mode, plot, iplot, download_plotlyjs
init_notebook_mode(connected = True)

import plotly.graph_objs as go

In [13]:
trace = go.Bar(x = districts.sort_values(['price']).name, 
               y = districts.sort_values(['price']).price.apply(lambda x: x/1000000))
layout = go.Layout(title = 'Precio Medio por Comuna', xaxis_title = 'Comuna', 
                   yaxis_title = 'Precio COP en (M)')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [14]:
trace = go.Bar(x = districts.sort_values(['surface_total']).name, 
               y = districts.sort_values(['surface_total']).surface_total)
layout = go.Layout(title = 'Superfcie Total Media por Comuna', xaxis_title = 'Comuna', 
                   yaxis_title = 'Superficie Total (m^2)')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [15]:
trace1 = go.Bar(x = districts.sort_values(['price']).name, 
               y = districts.sort_values(['price']).price.apply(lambda x: x/1000000),
               name = 'Precio COP (M)', marker_color = '#0b31db')
trace = go.Bar(x = districts.sort_values(['surface_total']).name, 
                y = districts.sort_values(['surface_total']).surface_total,
                name = 'Superficie Total', marker_color = '#db700b')
layout = go.Layout(title = 'Precio y Superficie total Media por Comuna', xaxis_title = 'Comuna', barmode = 'group')
fig = go.Figure(data = [trace, trace1], layout = layout)
fig.add_trace(go.Scatter(x = districts.sort_values(['surface_total']).name, 
                         y = districts.sort_values(['surface_total']).price.apply(lambda x: x/1000000),
                         mode = 'lines+markers',
                         name = 'Precio COP (M)'))
fig.add_trace(go.Scatter(x = districts.sort_values(['surface_total']).name,
                         y = districts.sort_values(['surface_total']).surface_total,
                         mode = 'lines+markers',
                         name = 'Superficie Total'))
iplot(fig)

In [16]:
trace = go.Bar(x = districts.sort_values(['bedrooms']).name, y = districts.sort_values(['bedrooms']).bedrooms)
layout = go.Layout(title = 'Numero Medio de Cuartos por Comuna', xaxis_title = 'Comuna', yaxis_title = 'Cuartos')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [17]:
trace = go.Bar(x = districts.sort_values(['bathrooms']).name, y = districts.sort_values(['bathrooms']).bathrooms)
layout = go.Layout(title = 'Numero Medio de Baños por Comuna', xaxis_title = 'Comuna', yaxis_title = 'Baños')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [51]:
trace1 = go.Bar(x = districts.sort_values(['bedrooms']).name, 
                y = districts.sort_values(['bedrooms']).bathrooms, name = 'Baños', marker_color = '#db700b')
trace = go.Bar(x = districts.sort_values(['bedrooms']).name, 
               y = districts.sort_values(['bedrooms']).bedrooms, name = 'Cuartos')
layout = go.Layout(title = 'Numero de Medio de Baños y Cuartos (Orden por Precio)', 
                   xaxis_title = 'Comuna', barmode = 'group')
fig = go.Figure(data = [trace, trace1], layout = layout)
fig.add_trace(go.Scatter(x = districts.sort_values(['bedrooms']).name, 
                         y = districts.sort_values(['bedrooms']).bathrooms,
                         mode = 'lines+markers',
                         name = 'Baños'))
fig.add_trace(go.Scatter(x = districts.sort_values(['bedrooms']).name,
                         y = districts.sort_values(['bedrooms']).bedrooms,
                         mode = 'lines+markers',
                         name = 'Cuartos'))
fig.add_trace(go.Scatter(x = districts.sort_values(['bedrooms']).name,
                         y = districts.sort_values(['bedrooms']).price.apply(lambda x: x/100000000),
                         mode = 'lines+markers',
                         name = 'Precio COP (100M)',
                         marker_color = 'black'))
iplot(fig)

# Observaciones
 
## Superficie total
 - Es probable que las viviendas que sean catalogadas como casas en Santa Elena, en realidad sean fincas, debido a su alto precio medio y su alta suferpeficie total media.
 - Hay comunas en las cuales la superficie total de no sigue la misma tendencia que el precio necesariamente. Esto puede ocurrir debido a que la zona efectivamente infla los precios de las viviendas que se encuentrar en dichos distritos, y los precios en esta zona efectivamente son delimitados por su sector.
 - Los picos de la anomalia mencionada previamentre corresponden a las comunas: LAURELES, LA CANDELARIA, GUAYABAL, VILLA HERMOSA, SAN ANTONIO DE PRADO, EL POBLADO y SANTA ELENA.
 - Los bajos corresponden a: ARANJUEZ, MANRIQUE, POPULAR y SAN JAVIER.

## Cuartos y Baños
 - El precio tampoco sigue la misma tendencia del numero medio de cuartos y baños por comuna. 
 - Cuando se organiza con respecto a los baños, hay picos en: VILLA HERMOSA, GUAYABAL, LA CANDELARIA, LAURELES, BUENOS AIRES, SAN ANTONIO DE PRADO, EL POBLADO Y SANTA ELENA.
 - Los bajos corresponden a: PALMITAS, POPULAR, SAN CRISTOBAL, Y EL DOCE DE OCTUBRE. 
 - Cuando se organizan con respecto al numero medio de cuartos el precio no sigue una tendencia clara en ninguna parte de la grafica, a diferencia de las graficas de las superficie total media y el numero medio de baños, donde habia una clara relacion entre estas dos variables al principio de la grafica. 

# Santa Elena

In [20]:
# Filtering
santa_elena = df_real.loc[df_real['SANTA ELENA'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
santa_elena.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,8.100000e+01,81.000000
mean,75.555556,-75.551486,6.202292,3.864198,5.086420,627.209877,1.852837e+09,0.123457
std,62.424955,0.007957,0.016980,0.862454,1.797203,600.356909,1.140980e+09,0.331010
min,3.000000,-75.559286,6.179083,2.000000,1.000000,96.000000,2.800000e+08,0.000000
25%,32.000000,-75.555849,6.187518,3.000000,4.000000,237.000000,7.800000e+08,0.000000
50%,76.000000,-75.552002,6.197128,4.000000,5.000000,496.000000,1.700000e+09,0.000000
75%,91.000000,-75.547997,6.224320,4.000000,7.000000,700.000000,2.700000e+09,0.000000
max,364.000000,-75.498001,6.226688,6.000000,8.000000,2700.000000,4.300000e+09,1.000000


In [ ]:
apto_elena = santa_elena[santa_elena['Apartamento'] == 1]
casa_elena = santa_elena[santa_elena['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_elena.shape[0])
print('Numero de casas: ', casa_elena.shape[0])

In [54]:
trace1 = go.Scatter(x = apto_elena.surface_total, y = apto_elena.price, mode = 'markers', name = 'Apartamentos',
                   marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_elena.surface_total, y = casa_elena.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (SANTA ELENA)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones 
 - Claramente la superficie total de incrementa de manera notoria el precio de las viviendas.
 - Si se compara con la superficie total de EL POBLADO, comuna cuya superficie total es la segunda mayor, despues de esta (SANTA ELENA), en general, las superficies totales son bastante grandes, lo que puede indicar que efectivamemnte la mayoria de estas viviendas pueden ser fincas